In [415]:
import pandas as pd
import numpy as np

Input will be 2 variables:
* Acquiror ticker symbol/ acquiror name
* Announcement date

# Get data

## Stock data

In [416]:
# access Manon's dictionary to get 15y of stock data of company

In [515]:
import requests

url = "http://iex.lewagon.com/stable/stock/aapl/chart/max" # max accesses past 15 y
data = requests.get(url)#.json()

In [519]:
data.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
comp = pd.DataFrame(data)

# transform date and set as index
comp['date'] = pd.to_datetime(comp['date'], format="%Y-%m-%d")
comp.set_index('date', inplace=True)

#comp = comp[['close']]

comp.head()

## Market data

In [509]:
# access csv file of market data
market = pd.read_csv('../raw_data/210906_S&P500_SPX.csv')

market.head()

,Exchange Date,Close,%Chg,Open,Volume
0,03-Jan-2005,"1,202.08",NaN,"1,211.92",0.0
1,04-Jan-2005,"1,188.05",-1.17%,"1,202.08",0.0
2,05-Jan-2005,"1,183.74",-0.36%,"1,188.05",0.0
3,06-Jan-2005,"1,187.89",+0.35%,"1,183.74",0.0
4,07-Jan-2005,"1,186.19",-0.14%,"1,187.89",0.0


In [510]:
from datetime import datetime, timedelta

# transform date and set as index
to_date = lambda x: datetime.strptime(x, '%d-%b-%Y')
market['Exchange Date'] = market['Exchange Date'].apply(to_date)
market.set_index('Exchange Date', inplace=True)

In [511]:
# transform closing price into float
rem_com = lambda x: x.replace(',', '')
market["Close"] = market['Close'].apply(rem_com).astype(float)
market["Open"] = market['Open'].apply(rem_com).astype(float)
market = market[['Close', 'Open']]

market.head()

,Close,Open
Exchange Date,,
2005-01-03,1202.08,1211.92
2005-01-04,1188.05,1202.08
2005-01-05,1183.74,1188.05
2005-01-06,1187.89,1183.74
2005-01-07,1186.19,1187.89


# Calculate returns

## Stock

In [497]:
comp_returns = comp.pct_change() # calculates percentage change of each day
# comp_returns = comp_returns[['close']]
# comp_returns.columns = ['comp_return']

comp_returns.head()

,close
date,
2006-12-29,NaN
2007-01-03,-0.012244
2007-01-04,0.022186
2007-01-05,-0.007126
2007-01-08,0.004938


In [438]:
# check
np.round(2.9929 / 3.03 - 1, 6) == -0.012244

True

## Market

In [512]:
market_returns = market.pct_change()
# market_returns = 
market_returns['Close']
# market_returns.columns = ['market_return']

# market_returns.head()

Exchange Date
2005-01-03         NaN
2005-01-04   -0.011671
2005-01-05   -0.003628
2005-01-06    0.003506
2005-01-07   -0.001431
                ...   
2021-08-30    0.004307
2021-08-31   -0.001349
2021-09-01    0.000312
2021-09-02    0.002843
2021-09-03   -0.000335
Name: Close, Length: 4198, dtype: float64

## Market adjusted model

In [444]:
# merge dataframes
market_adj = comp_returns.join(market_returns)

# company return - market return
market_adj['abnormal_return'] = market_adj.comp_return - market_adj.market_return

market_adj.head()

,comp_return,market_return,abnormal_return
date,,,
2006-12-29,NaN,-0.004513,NaN
2007-01-03,-0.012244,-0.001199,-0.011046
2007-01-04,0.022186,0.001228,0.020958
2007-01-05,-0.007126,-0.006085,-0.001041
2007-01-08,0.004938,0.002220,0.002718


In [484]:
market_adj['abnormal_return'].loc[(market_adj.index >= start_day) & (market_adj.index <= end_day)].sum()

0.011636883137329135

# CAR

In [445]:
from pandas.tseries.offsets import BDay

In [458]:
# Set event horizon
def calculate_car(ticker, announcement_date, start=1, end=1):
    
    # transform into business days
    start_day = announcement_date - BDay(start)

    # define function that takes care of non-trading days (taking the later trading day)
    if announcement_date.weekday() > 4:
        end_day = announcement_date + BDay(2) # otherwise will just take event horizon -1 into consideration
        
    else:
        end_day = announcement_date + BDay(end)
        
    # get data
    
    
    
    # calculate CAR
    car = market_adj.loc[(market_adj.index >= start_day) & (market_adj.index <= end_day)]['abnormal_return'].sum()

    return car

In [463]:
ann = pd.to_datetime('2021-08-14', format="%Y-%m-%d")

In [465]:
# transform into business days
start_day = ann - BDay(1)

# define function that takes care of non-trading days (taking the later trading day)
if ann.weekday() > 4:
    end_day = ann + BDay(2) # otherwise will just take event horizon -1 into consideration

else:
    end_day = ann + BDay(1)

In [466]:
print(start_day, end_day)

2021-08-13 00:00:00 2021-08-17 00:00:00


In [467]:
end_day = ann + BDay(2) if ann.weekday() > 4 else ann + BDay(1)

In [468]:
end_day

Timestamp('2021-08-17 00:00:00')